In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
# read train.csv
df = pd.read_csv("../input/aaaaaaaaaa/train.csv")

In [ ]:
# initialize labels_list, labels_encode
labels_list = []
labels_encode = {}
count = 0
for i in range(df.shape[0]):
    if df["label"][i] not in labels_list:
        labels_list.append(df["label"][i])
        labels_encode[df["label"][i]] = count
        count+=1

In [ ]:
labels_list = ['3_24+10', '3_24+30', '3_24+5', '3_24+60', '3_24+70', '3_24+90', '3_24+110', '3_24+20', '3_24+40', '3_24+50', '3_24+80', '1_12_1', '1_12_2', '1_13', '1_14', '1_19', '1_24', '1_26', '1_27', '3_21', '3_31', '3_33', '4_4_1', '4_4_2', '4_5_2', '4_5_4', '4_5_5', '4_8_5', '4_8_6', '5_17', '6_2+50', '6_2+70', '6_2+30', '6_2+40', '6_2+60', '6_2+80', '6_7', '7_1', '7_11', '7_13', '7_14', '7_2', '7_4', '7_7', '7_9', 'smoke', 'unknown', '1_11_1', '1_11_2', '1_15', '1_16', '1_18', '1_20_1', '1_22', '1_25', '1_28', '1_29', '1_30', '1_8', '2_3_1', '2_3_L', '2_3_R', '2_6', '2_7', '3_15', '3_17', '3_20', '3_25+70', '3_25+20', '3_25+30', '3_25+40', '3_25+50', '3_25+5', '3_25+60', '3_6', '4_1_6', '4_2_1', '4_2_2', '5_15_5', '6_3_1', '7_3', '7_6', '1_17', '3_16', '5_15_3', '5_20', '7_12', '1_31', '3_10', '3_19', '3_2', '3_5', '3_7', '3_9', '4_1_2_1', '4_1_3_1', '4_5_1', '4_5_6', '4_8_1', '4_8_2', '4_8_3', '5_1', '5_11_1', '5_12_1', '5_13_1', '5_13_2', '5_14_1', '5_14_2', '5_14_3', '5_2', '5_23_2', '5_24_2', '5_3', '5_4', '5_8', '7_5', '3_32', '7_18', '1_2', '1_33', '1_7', '2_4', '3_18_1', '3_18_2', '3_8', '4_1_2', '4_1_3', '5_14', '6_15_2', '6_15_3', '6_6', '6_8_1', '1_1', '1_20_2', '1_20_3', '1_21', '1_23', '1_5', '2_1', '2_2', '2_5', '3_1', '3_26', '3_27', '3_28', '3_29', '3_30', '4_1_1', '4_1_4', '4_1_5', '4_2_3', '4_3', '4_8_4', '5_16', '5_18', '5_19', '5_21', '5_22', '5_5', '5_6', '5_7_1', '5_7_2', '5_9', '6_15_1', '6_16', '6_4', '6_8_2', '6_8_3', '5_29', '5_31+10', '5_31+20', '5_31+30', '5_31+40', '5_31+5', '5_31+50', '5_32', '5_33', '1_6', '5_15_2+2', '5_15_2+1', '5_15_2+3', '5_15_2+5']

In [ ]:
# load all preprocessed images
images = []
root_dir = "../input/encoded-imgs/Autoencoder/"
labels = []
X = []

for i in tqdm(range(10000)):
    img_path = root_dir + df["filename"][i][13:]
    X.append(cv2.resize(cv2.imread(img_path),dsize=(80,80)))
    labels.append(labels_encode[df["label"][i]])
    
X = np.array(X)
y = np.array(labels)
Y = to_categorical(y, num_classes=182)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2,stratify = Y, shuffle = True, random_state = 42)

In [ ]:
aug = ImageDataGenerator(brightness_range=[0.2, 1.8],
                         rotation_range=10, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.15, 
                         zoom_range=0.15,
                         rescale=1/255.0,
                         fill_mode="nearest")

In [ ]:
def scheduler(epoch):
    if epoch<= WARMUP_EPOCH:
        lr = INITIAL_LEARNINGRATE * epoch/WARMUP_EPOCH
    
    else:
        lr = INITIAL_LEARNINGRATE * DECAY_RATE**(epoch-WARMUP_EPOCH)
    
    return lr

In [ ]:
def BuildModel(ModelName):
    print('--------------Building The Model...--------------')
    base_model = tf.keras.applications.InceptionResNetV2(include_top=False,
                                                         weights='imagenet',
                                                         input_shape=(80,80,3))
    base_model.trainable = True
    print("\nNumber of layers in the base model: ", len(base_model.layers))
    x = base_model.output
    #x = tf.keras.layers.GlobalMaxPooling2D()(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dense(512,activation='relu')(x)
    out = tf.keras.layers.Dense(182, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=out)

    print('\n--------------Done!--------------')
    return model

In [ ]:
def ModelDeployment(model, LearningRate, ModelName,  NbOfEpoch, StepsPerEpoch, ValidationSteps):
    ## Freeze all the layers before the `fine_tune_at` layer
    #for layer in base_model.layers[:fine_tune_at]:
    #    layer.trainable =  False
    print('--------------Deploying the Model...--------------')
    model.compile(loss = 'categorical_crossentropy', 
                  optimizer=Adam(lr=LearningRate),
                  metrics='accuracy')
    monitor = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                               min_delta=0.0001, 
                                               patience=5, 
                                               verbose=1, 
                                               mode='min',
                                               restore_best_weights=True)
    
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                        factor=0.5,
                                                        patience=2,
                                                        verbose=1,
                                                        mode='min',
                                                        min_delta=0.0001,
                                                        cooldown=0,
                                                        min_lr=MIN_LR,)

    filepath = ModelName + "-{epoch:02d}.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, 
                                                    monitor='loss', 
                                                    verbose=1, 
                                                    save_best_only=True, 
                                                    save_weights_only=True, 
                                                    mode='min',
                                                    save_freq = 'epoch') 
    print('--------------Deployed Successfully--------------')
    print('--------------Training Begins--------------')
    
    history = model.fit(aug.flow(X_train, y_train, batch_size = 128), 
                            epochs = NbOfEpoch, 
                            steps_per_epoch = StepsPerEpoch, 
                            validation_data = (X_val,y_val), 
                            verbose = 1, 
                            validation_steps = ValidationSteps,
                            callbacks = [monitor, checkpoint, lr_scheduler]) #callbacks = [monitor,lr_scheduler]
    return history

In [ ]:
def HistoryPlotting(history):
    fig,ax = plt.subplots(figsize=(8,5))
    ax.plot(history.history['loss'],label='train',color='blue')
    ax.plot(history.history['val_loss'],label='val',color='green')
    ax.set_title('model loss',fontsize=14)
    ax.set_ylabel('loss')
    ax.set_xlabel('epoch')
    ax.legend(loc='upper right',fontsize=13)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
WIDTH = 80
BATCH_SIZE = 128
model_name = "InceptionResNetV2"

model = BuildModel(ModelName=model_name)
model.summary()

In [ ]:
STEPS_PER_EPOCH = len(y_train)//BATCH_SIZE
VALIDATION_STEPS = len(y_val)//BATCH_SIZE
LEARNING_RATE = 0.0002
NB_EPOCH = 1000 
MIN_LR = 1e-08

history = ModelDeployment(model = model, 
                          LearningRate = LEARNING_RATE, 
                          ModelName = model_name,
                          NbOfEpoch = NB_EPOCH,
                          StepsPerEpoch = STEPS_PER_EPOCH,
                          ValidationSteps = STEPS_PER_EPOCH)

In [ ]:
model.save("InceptionResNet.h5")

In [ ]:
#save history
his = pd.DataFrame({'loss': list(history.history["loss"]), 
                    'accuracy': list(history.history["accuracy"]), 
                    'val_loss': list(history.history["val_loss"]), 
                    "val_accuracy": list(history.history["val_accuracy"
                    ])})
his.to_csv('history.csv', index=True)

In [ ]:
HistoryPlotting(history)

In [ ]:
# re-evaluate

In [ ]:
scores = model.evaluate(X, Y)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
results = model.evaluate(X_val, y_val, batch_size=128)
print("test loss, test acc:", results)
y_pred = model.predict(X_val)
classification_report(y_val.argmax(axis=1), y_pred.argmax(axis=1), target_names=labels_list)

print(confusion_matrix(y_val.argmax(axis=1), y_pred.argmax(axis=1)))